# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from create_tables import create_database

In [21]:
try:
    # Connection to the database sparkifydb
    # psql -h 127.0.0.1 -d sparkifydb -U student -W
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    conn.set_session(autocommit=True)
    cur = conn.cursor()
except:
    # If the connection fails we attempt to create the database
    cur, conn = create_database()

In [3]:
# Helper methods

def run_query(cursor, query, print_results=True):
    '''
    This method executes the query provided
    '''
    
    cursor.execute(query)
    if print_results:
        rows = cursor.fetchall()
        for r in rows:
            print(r)
            
            
def get_columns(cursor, table, as_string=True):
    '''
    This returns the column names of the table.
    If as_string is set to True, the column names will be passed as
    an string with the fromat (col1, clo2, ...) to be used in SQL INSERT statements
    '''
    
    cursor.execute("SELECT * FROM {} LIMIT 0".format(table))
    colnames = [desc[0] for desc in cursor.description]
    
    if as_string:
        return "("+", ".join(map(str, colnames))+")"
    else:
        return colnames

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [6]:
song_files = get_files("./data/song_data/")

In [7]:
filepath = "/home/workspace/data/song_data/A/B/B/TRABBVJ128F92F7EAA.json"
df = pd.read_json(filepath, lines=True)
# To avoid problems ingesting NaN values into Postgres I use None since NULL does not work
# df = df.fillna('Null')
df = df.where(pd.notnull(df), None)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AREDL271187FB40F44,None,,None,Soul Mekanik,45.6616,1,SOPEGZN12AB0181B3D,Get Your Head Stuck On Your Neck,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
# get_columns helper function for getting the columns fo the table
song_columns = get_columns(cursor=cur, table="songs", as_string=False)
song_data = df[song_columns].values.tolist()[0]

#parsed_data = "("+", ".join(map(str, song_data))+")"
#parsed_data = parsed_data.replace("]","").replace("[","")
#print(parsed_data)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
# Move this into sql_queries.py
song_columns_str = get_columns(cursor=cur, table="songs")
data_format = ", ".join(map(str, ["%s"]*len(song_columns)))
song_table_insert = "INSERT INTO songs {} VALUES ({})".format(song_columns_str,\
                                                              data_format)
song_table_insert


'INSERT INTO songs (song_id, title, artist_id, year, duration) VALUES (%s, %s, %s, %s, %s)'

In [9]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

In [9]:
# method for insertion data capturing the above cells
# filepath set to "/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json" for testing

def insert_song_data(filepath,\
                     cursor,\
                     connection,\
                     table):
    
    '''
    This method extracts data from song_data and insert it into the target <table> 
    '''
    
    df = pd.read_json(filepath, lines=True)
    df = df.where(pd.notnull(df), None)
    data_columns = get_columns(cursor=cur, table=table, as_string=False)
    data = df[data_columns].values.tolist()[0]

    data_columns_str = get_columns(cursor=cursor, table=table)
    data_format = ", ".join(map(str, ["%s"]*len(data_columns)))
    data_table_insert = "INSERT INTO {} {} VALUES ({})".format(table,\
                                                               data_columns_str,\
                                                               data_format)
    
    cursor.execute(data_table_insert, data)
    connection.commit()

# As a test I set the path to "/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json"
# One can just loop over the files in "/home/workspace/data/song_data" to insert all the data
# Before executing this cell take into account that song_id is PRIMARY KEY 
insert_song_data(filepath="/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json",\
                 cursor=cur,\
                 connection=conn,\
                 table="songs")

IntegrityError: duplicate key value violates unique constraint "songs_pkey"
DETAIL:  Key (song_id)=(SOBCOSW12A8C13D398) already exists.


## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [32]:
# get_columns helper function for getting the columns fo the table
artist_columns = get_columns(cursor=cur, table="artists", as_string=False)
artist_data = df[artist_columns].values.tolist()[0]

KeyError: "['artist_id' 'artist_name' 'artist_location' 'artist_latitude'\n 'artist_longitude'] not in index"

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [50]:
# Move this into sql_queries.py

artist_columns_str = get_columns(cursor=cur, table="artists")
data_format = ", ".join(map(str, ["%s"]*len(artist_columns)))
artist_table_insert = "INSERT INTO artists {} VALUES ({})".format(artist_columns_str,\
                                                                  data_format)

In [40]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

DataError: invalid input syntax for integer: "NULL"
LINE 1: ...ES ('AREDL271187FB40F44', 'Soul Mekanik', 'NULL', 'NULL', 'N...
                                                             ^


Run `test.ipynb` to see if you've successfully added a record to this table.

In [10]:
insert_song_data(filepath="/home/workspace/data/song_data/A/B/B/TRABBBV128F42967D7.json",\
                 cursor=cur,\
                 connection=conn,\
                 table="artists")

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [5]:
log_files = get_files("./data/log_data/")

In [6]:
filepath = "/home/workspace/data/log_data/2018/11/2018-11-01-events.json"
df = pd.read_json(filepath, lines=True)
# To avoid problems ingesting NaN values into Postgres I use None since NULL does not work
# df = df.fillna('Null')
df = df.where(pd.notnull(df), None)
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [10]:
df.head(10)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,None,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1540919166796,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,None,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1540344794796,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.308,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,None,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1540344794796,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.039,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.188,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.421,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.267,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
8,Blue October / Imogen Heap,Logged In,Kaylee,F,7,Summers,241.397,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Congratulations,200,1541107493796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
9,Girl Talk,Logged In,Kaylee,F,8,Summers,160.156,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,Once again,200,1541107734796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [7]:
# Filter records by NextSong action
# "NextSong" is one of the "page" values
df = df[df['page'] == "NextSong"]
df.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.308,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1540344794796,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [8]:
# "ts" conversion into date-time
df["tsdt"] = pd.to_datetime(df["ts"], unit='ms')
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId', 'tsdt'],
      dtype='object')

In [23]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday 
# from the ts column and set time_data to a list containing these values in order
df["hour"] = df.tsdt.dt.hour
df["day"] = df.tsdt.dt.day
df["weekofyear"] = df.tsdt.dt.weekofyear
df["month"] = df.tsdt.dt.month
df["year"] = df.tsdt.dt.year
df["weekday"] = df.tsdt.dt.weekday
#time_data = (df.hour, df.day, df.weekofyear, df.month, df.weekday)
#column_labels = ("hour", "day", "weekofyear", "month", "weekday")

# the time table has the clolumns:
# start_time, hour, day, week, month, year, weekday
# Ordering the data accordingly and including start_time
time_data = (df.tsdt, df.hour, df.day, df.weekofyear, df.month, df.year, df.weekday)
column_labels = ("tsdt", "hour", "day", "weekofyear", "month", "year", "weekday")


In [24]:
# Create a dataframe, time_df, containing the time data 
# for this file by combining column_labels and time_data 
# into a dictionary and converting this into a dataframe

time_dic = dict(zip(column_labels, time_data))
time_df = pd.DataFrame.from_dict(time_dic) 
time_df

,tsdt,hour,day,weekofyear,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3
8,2018-11-01 21:24:53.796,21,1,44,11,2018,3
9,2018-11-01 21:28:54.796,21,1,44,11,2018,3
10,2018-11-01 21:42:00.796,21,1,44,11,2018,3
12,2018-11-01 21:52:05.796,21,1,44,11,2018,3
13,2018-11-01 21:55:25.796,21,1,44,11,2018,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [25]:
# move time_table_insert into sql_queries.py

# The table time has the columns:
# start_time, hour, day, week, month, year, weekday

time_columns = get_columns(cursor=cur, table="time", as_string=False)
time_columns_str = get_columns(cursor=cur, table="time")
data_format = ", ".join(map(str, ["%s"]*len(time_columns)))

time_table_insert = "INSERT INTO {} {} VALUES ({})".format("time",\
                                                           time_columns_str,\
                                                           data_format)
time_table_insert

'INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)'

In [27]:
print(time_dic.keys())


for i, row in time_df.iterrows():
    print(time_table_insert)
    print(list(row))
    cur.execute(time_table_insert, list(row))
    conn.commit()

dict_keys(['tsdt', 'hour', 'day', 'weekofyear', 'month', 'year', 'weekday'])
INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)
[Timestamp('2018-11-01 21:01:46.796000'), 21, 1, 44, 11, 2018, 3]
INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)
[Timestamp('2018-11-01 21:05:52.796000'), 21, 1, 44, 11, 2018, 3]
INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)
[Timestamp('2018-11-01 21:08:16.796000'), 21, 1, 44, 11, 2018, 3]
INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)
[Timestamp('2018-11-01 21:11:13.796000'), 21, 1, 44, 11, 2018, 3]
INSERT INTO time (start_time, hour, day, week, month, year, weekday) VALUES (%s, %s, %s, %s, %s, %s, %s)
[Timestamp('2018-11-01 21:17:33.796000'), 21, 1, 44, 11, 2018, 3]
INSERT INTO time (start_time, hour, day, week, month, year, weekday)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [20]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.